In [23]:
timestamp = "2023-04-17 22:00" # Enter timestamp in UTC in this format
api_key = "xxxxxxxx-yyyy-zzzz-aaaa-bbbbbbbbbbbb" # Enter your API key
garden_point_contract = "0xFeffc0E5C9575576C1922978102afa2D803Dc93F" # Enter garden point's contract
ch_proxy = "0x0b2e0bdaffd0881988f37057104977c9206fe481" # Enter cyberhornets' proxy contract on kometh
ch_comics = "0x303Fd791674D67e1288f769B54699c170eEdE5e7" # Enter cyberhornets' comic contract 

In [ ]:
# format for api_key
# api_key = "xxxxxxxx-yyyy-zzzz-aaaa-bbbbbbbbbbbb"

In [ ]:
pip install shroomdk

In [2]:
import pandas as pd 

In [3]:
from shroomdk import ShroomDK

# Initialize `ShroomDK` with your API Key
sdk = ShroomDK(api_key)


# First query is to get a block number for snapshot that corresponds to snapshot time
sql_blocknumber = f"""
    SELECT
        block_timestamp, 
        block_number
    FROM ethereum.core.ez_nft_transfers
    WHERE date_trunc('minute', block_timestamp) = '{timestamp}'
    ORDER BY block_timestamp DESC
    LIMIT 1
"""

query_result_timestamp = sdk.query(sql_blocknumber)

for record in query_result_timestamp.records:
    snapshot = record['block_number']

In [4]:
# Parameters can be passed into SQL statements 
# via native string interpolation
# Second query is to obtain token wallet balances during a particular block 

my_address = garden_point_contract # garden point contract

sql = f"""
    WITH wallets as(
    SELECT 
        nft_to_address as wallet, 
        max(block_number) as last_transfer 
    FROM ethereum.core.ez_nft_transfers 
    WHERE nft_address = LOWER('{my_address}')
    GROUP BY wallet
    ),

    first_data as(
    SELECT
        nft_to_address as owner,
        tokenid
    FROM ethereum.core.ez_nft_transfers
    WHERE nft_address = LOWER('{my_address}')
    AND block_number <= '{snapshot}'
    QUALIFY RANK() OVER (
        PARTITION BY tokenid 
        ORDER BY block_number DESC, event_index DESC) = 1        
    )
    SELECT *
    FROM first_data
    ORDER BY owner
    
"""

# Run the query against Flipside's query engine 
# and await the results
query_result_set = sdk.query(sql)

In [18]:
# third sql is to retrieve the minters of cyberhornets

sql_3 = f"""
    with chc_mints as (
    select
      block_timestamp, 
      block_number, 
      event_type,
      nft_address,
      nft_from_address as sender, 
      nft_to_address as wallet, 
      tokenid,
      erc1155_value 
    from ethereum.core.ez_nft_transfers
    where nft_from_address = LOWER('{ch_proxy}')
    and block_number <= '{snapshot}'
    )

    , chc_transactions as (
    select 
      block_timestamp, 
      block_number, 
      event_type, 
      nft_address, 
      nft_from_address as sender,
      nft_to_address as recipient, 
      tokenid, 
      erc1155_value
    from ethereum.core.ez_nft_transfers 
    where nft_address = LOWER('{ch_comics}') 
    )

    select wallet, sum(erc1155_value) as minted from chc_mints
    group by wallet
    order by minted DESC
    
"""

# Run the query against Flipside's query engine 
# and await the results
query_result_set_3 = sdk.query(sql_3)

In [19]:
# create empty data frame container for garden point rarity holder
data = []

# convert sql run result into pandas dataframe
for record in query_result_set.records:
    owner = record['owner']
    tokenid = record['tokenid']
    data.append({'owner': owner, 'token_id': tokenid})
df = pd.DataFrame(data)

# convert token id data type from object/string to integer
df['token_id'] = df['token_id'].astype(int)

In [20]:
# create empty data frame container for cyber hornets minters

data_3 = []

for record in query_result_set_3.records:
    wallet = record['wallet']
    minted = record['minted']
    data_3.append({'wallet': wallet, 'minted': minted})
df_3 = pd.DataFrame(data_3)

# data frame for cyber hornets minters through kometh

df_3['minted'] = df_3['minted'].astype(int)



In [6]:
# import garden point token metadata from github repository

token_metadata_url = 'https://raw.githubusercontent.com/fcitra/GP-snapshot/main/GardenPoint%20Covers.csv'
token_metadata_df = pd.read_csv(token_metadata_url)

In [7]:
# merge token holders' snapshot with token metadata
merged_df = pd.merge(df, token_metadata_df, on ='token_id', how = 'left')

# fill missing metadata value for new tokens minted from burning event 
merged_df['Metadata'].fillna('RARE', inplace = True)

# group token quantity based on holder wallet and rarity type
grouped_df = merged_df.groupby(['owner', 'Metadata'])['Metadata'].size()

# unstack grouped rows into columns
df_unstack = grouped_df.unstack()

# change NaN value into 0 
df_unstack = df_unstack.fillna(0)

In [ ]:
# create file csv file name for garden point holders
csv_name = 'snapshot_rarity_block_' + str(snapshot) + '.csv'

# create file csv for ch minters
csv_minters_name = 'ch_minters_block_' + str(snapshot) + '.csv'

# export dataframe to csv file 
df_unstack.to_csv(csv_name, index = True)

# export dataframe to csv file 
df_3.to_csv(csv_minters_name, index = True)